In [1]:
import datetime
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
from dataset import Dataset

In [167]:
klines = Dataset().get_data('SOLUSDT', '15m', start_date='2024-1-1')
klines = klines[klines.columns[:4]]

In [168]:
klines

,open,high,low,close
close_time,,,,
2024-01-01 00:15:00,101.72,102.79,101.56,102.36
2024-01-01 00:30:00,102.35,102.53,101.69,102.07
2024-01-01 00:45:00,102.05,102.50,101.63,102.18
2024-01-01 01:00:00,102.16,102.28,101.93,101.96
2024-01-01 01:15:00,101.96,102.56,101.51,102.40
...,...,...,...,...
2025-01-25 19:30:00,258.17,259.87,258.16,259.27
2025-01-25 19:45:00,259.28,259.76,258.39,258.40
2025-01-25 20:00:00,258.39,258.50,257.37,258.50


In [169]:
sum(klines.index.time == datetime.time(0, 0))

390

In [170]:
klines_daily = Dataset().get_data('SOLUSDT', '1d', start_date='2023-12-30')

In [171]:
klines_daily

,open,high,low,close,volume,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume
close_time,,,,,,,,,
2023-12-31,106.17,107.47,100.62,101.90,4.178340e+06,4.323704e+08,527519.0,2025406.370,2.096545e+08
2024-01-01,101.90,105.21,99.54,101.72,3.821991e+06,3.931549e+08,490129.0,1891118.520,1.946794e+08
2024-01-02,101.72,109.93,101.44,109.91,4.403310e+06,4.647638e+08,563530.0,2246699.810,2.372047e+08
2024-01-03,109.93,116.95,106.02,106.73,7.831366e+06,8.727018e+08,1029366.0,4008477.700,4.468272e+08
2024-01-04,106.72,109.90,85.00,98.52,1.222106e+07,1.229681e+09,1428502.0,6232536.620,6.274338e+08
...,...,...,...,...,...,...,...,...,...
2025-01-22,242.18,260.06,230.00,250.33,7.882737e+06,1.907066e+09,6181509.0,3813585.870,9.230475e+08
2025-01-23,250.33,272.03,248.43,257.36,6.223428e+06,1.612463e+09,4791221.0,3103831.608,8.048723e+08
2025-01-24,257.37,258.48,242.19,253.11,6.802763e+06,1.698329e+09,5535365.0,3354267.435,8.374085e+08


In [173]:
rr = []
hold = 360  # minutes

for t in klines_daily.index[1:-1]:
    t += datetime.timedelta(minutes=15)
    h, l = klines.loc[t].high, klines.loc[t].low
    day = klines.loc[t + datetime.timedelta(minutes=5):t + datetime.timedelta(minutes=1435)]
    
    break_candle_1 = day[(day.close > h) | (day.close < l)].iloc[0]
    day = day.loc[break_candle_1.name:]
    entry = break_candle_1.close
    exit_time = t + datetime.timedelta(minutes=hold)
    
    if entry >= h:  # long
        exit_candle = day[(day.low <= l) | (day.index == exit_time)].iloc[0]
        if (exit_candle.name == exit_time) and (exit_candle.close > entry):
            rr.append((exit_candle.close - entry) / (entry - l))
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            exit_time += datetime.timedelta(minutes=hold)
            if entry >= h:  # long
                exit_candle = day[(day.low <= l) | (day.index == exit_time)].iloc[0]
                if (exit_candle.name == exit_time) and (exit_candle.close > entry):
                    rr.append((exit_candle.close - entry) / (entry - l))
                else:
                    rr.append(-1)
            else:  # short
                exit_candle = day[(day.high >= h) | (day.index == exit_time)].iloc[0]
                if (exit_candle.name == exit_time) and (exit_candle.close < entry):
                    rr.append((entry - exit_candle.close) / (h - entry))
                else:
                    rr.append(-1)
                    
    if entry <= l:  # short
        exit_candle = day[(day.high >= h) | (day.index == exit_time)].iloc[0]
        if (exit_candle.name == exit_time) and (exit_candle.low < entry):
            rr.append((entry - exit_candle.close) / (h - entry))
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            exit_time += datetime.timedelta(minutes=hold)
            if entry >= h:  # long
                exit_candle = day[(day.low <= l) | (day.index == exit_time)].iloc[0]
                if (exit_candle.name == exit_time) and (exit_candle.close > entry):
                    rr.append((exit_candle.close - entry) / (entry - l))
                else:
                    rr.append(-1)
            else:  # short
                exit_candle = day[(day.high >= h) | (day.index == exit_time)].iloc[0]
                if (exit_candle.name == exit_time) and (exit_candle.close < entry):
                    rr.append((entry - exit_candle.close) / (h - entry))
                else:
                    rr.append(-1)
                    
print(sum(rr))

-319.3539803134629


In [174]:
rr = []
target_rr = 2
hold = 180  # minutes

for t in klines_daily.index[1:-1]:
    yh, yl = klines_daily.loc[t - datetime.timedelta(days=1)].high, klines_daily.loc[t - datetime.timedelta(days=1)].low
    t += datetime.timedelta(minutes=15)
    h, l = klines.loc[t].high, klines.loc[t].low
    day = klines.loc[t + datetime.timedelta(minutes=5):t + datetime.timedelta(minutes=1435)]
    
    break_candle_1 = day[(day.close > h) | (day.close < l)].iloc[0]
    day = day.loc[break_candle_1.name:]
    entry = break_candle_1.close
    exit_time = t + datetime.timedelta(minutes=hold)
    
    if entry >= h:  # long
        target = entry + (entry - l) * target_rr
        exit_candle = day[(day.high >= target) | (day.low <= l) | (day.index == exit_time)].iloc[0]
        if exit_candle.high >= target:
            rr.append(target_rr)
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            exit_time += datetime.timedelta(minutes=hold)
            if entry >= h:  # long
                target = entry + (entry - l) * target_rr
                exit_candle = day[(day.high >= target) | (day.low <= l) | (day.index == exit_time)].iloc[0]
                if exit_candle.high >= target:
                    rr.append(target_rr)
                else:
                    rr.append(-1)
            else:  # short
                target = entry - (h - entry) * target_rr
                exit_candle = day[(day.low <= target) | (day.high >= h) | (day.index == exit_time)].iloc[0]
                if exit_candle.low <= target:
                    rr.append(target_rr)
                else:
                    rr.append(-1)
                    
    elif entry <= l:  # short
        target = entry - (h - entry) * target_rr
        exit_candle = day[(day.low <= target) | (day.high >= h) | (day.index == exit_time)].iloc[0]
        if exit_candle.low <= target:
            rr.append(target_rr)
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            exit_time += datetime.timedelta(minutes=hold)
            if entry >= h:  # long
                target = entry + (entry - l) * target_rr
                exit_candle = day[(day.high >= target) | (day.low <= l) | (day.index == exit_time)].iloc[0]
                if exit_candle.high >= target:
                    rr.append(target_rr)
                else:
                    rr.append(-1)
            else:  # short
                target = entry - (h - entry) * target_rr
                exit_candle = day[(day.low <= target) | (day.high >= h) | (day.index == exit_time)].iloc[0]
                if exit_candle.low <= target:
                    rr.append(target_rr)
                else:
                    rr.append(-1)
                    
print(sum(rr))

IndexError: single positional indexer is out-of-bounds

In [176]:
rr = []

for t in klines_daily.index[1:-1]:
    yh, yl = klines_daily.loc[t - datetime.timedelta(days=1)].high, klines_daily.loc[t - datetime.timedelta(days=1)].low
#     yh *= 0.85
#     yl *= 1.15
    t += datetime.timedelta(minutes=15)
    h, l = klines.loc[t].high, klines.loc[t].low
    day = klines.loc[t + datetime.timedelta(minutes=5):t + datetime.timedelta(minutes=1435)]
    
    break_candle_1 = day[(day.close > h) | (day.close < l)].iloc[0]
    day = day.loc[break_candle_1.name:]
    entry = break_candle_1.close
    
    if entry >= h and yh > entry:  # long
        exit_candle = day[(day.high >= yh) | (day.low <= l) | (day.index == day.index[-1])].iloc[0]
        
        if exit_candle.high >= yh:
            rr.append((yh - entry) / (entry - l))
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            if entry >= h and yh > entry:  # long
                exit_candle = day[(day.high >= yh) | (day.low <= l) | (day.index == day.index[-1])].iloc[0]
                if exit_candle.high >= yh:
                    rr.append((yh - entry) / (entry - l))
                else:
                    rr.append(-1)
            elif entry <= l and yl < entry:  # short
                exit_candle = day[(day.low <= yl) | (day.high >= h) | (day.index == day.index[-1])].iloc[0]
                if exit_candle.low <= yl:
                    rr.append((entry - yl) / (h - entry))
                else:
                    rr.append(-1)
                    
    elif entry <= l and yl < entry:  # short
        exit_candle = day[(day.low <= yl) | (day.high >= h) | (day.index == day.index[-1])].iloc[0]
        if exit_candle.low <= yl:
            rr.append((entry - yl) / (h - entry))
        else:
            rr.append(-1)
            day = day.loc[exit_candle.name:]
            break_candle_2 = day[(day.close > h) | (day.close < l)].iloc[0]
            day = day.loc[break_candle_2.name:]
            entry = break_candle_2.close
            if entry >= h and yh > entry:  # long
                exit_candle = day[(day.high >= yh) | (day.low <= l) | (day.index == day.index[-1])].iloc[0]
                if exit_candle.high >= yh:
                    rr.append((yh - entry) / (entry - l))
                else:
                    rr.append(-1)
            elif entry <= l and yl < entry:  # short
                exit_candle = day[(day.low <= yl) | (day.high >= h) | (day.index == day.index[-1])].iloc[0]
                if exit_candle.low <= yl:
                    rr.append((entry - yl) / (h - entry))
                else:
                    rr.append(-1)
                    
print(sum(rr))

plt.figure()
plt.plot(np.cumsum(rr))
plt.show()

IndexError: single positional indexer is out-of-bounds